In [ ]:
from pymongo import MongoClient
from confluent_kafka import Consumer, KafkaError

# MongoDB connection details
uri = 'mongodb://localhost:27017'
db_name = 'consumer1databse'
collection_name = 'coleection1'

# Kafka consumer configuration
conf = {'bootstrap.servers': 'localhost:9092', 'group.id': 'myconsumer'}

# Connect to MongoDB
client = MongoClient(uri)
db = client[db_name]
collection = db[collection_name]

# Create Kafka consumer
consumer = Consumer(conf)

# Subscribe to Kafka topics
consumer.subscribe(['mytopic'])

def save_data(data):
    try:
        collection.insert_one(data)
        print("Data saved successfully!")
    except Exception as e:
        print("Error saving data:", e)

def process_data(data):
    save_data(data)

try:
    while True:
        msg = consumer.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition, no more messages
                continue
            else:
                # Error occurred
                print(f"Kafka error: {msg.error()}")
                break
        else:
            # Process and save message data
            data = msg.value().decode('utf-8')  # Decode message bytes to string
            process_data(data)
except KeyboardInterrupt:
    pass

finally:
    # Close Kafka consumer
    consumer.close()
    # Close MongoDB connection
    client.close()
